In [1]:
import nltk
nltk.download('punkt')

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import pandas as pd
from openai import OpenAI
# from google.colab import userdata
import re
import fitz

import fitz
import re

from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    true = labels.argmax(axis=-1)
    acc = accuracy_score(true, preds)
    return {"accuracy": acc}

def extract_questions_from_pdf(pdf_path, max_questions=34):
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])

    range_tags = re.findall(r'\[(\d+)[~～](\d+)\]', full_text)
    paragraph_ranges = [(int(start), int(end)) for start, end in range_tags]

    raw_blocks = re.split(r'\n?\s*(\d{1,2})[.)번]?\s+', full_text)

    questions = []
    current_shared_paragraph = ""
    current_range = None
    prev_text = raw_blocks[0]

    for i in range(1, len(raw_blocks) - 1, 2):
        number = int(raw_blocks[i])
        if number > max_questions:
            break  # 34번까지만

        body = raw_blocks[i + 1]
        in_shared_range = False
        for r in paragraph_ranges:
            if r[0] <= number <= r[1]:
                in_shared_range = True
                if current_range != r:
                    pattern = re.compile(rf'\[{r[0]}[~～]{r[1]}\](.*?)(?=\n\s*{r[0]}[.)번])', re.DOTALL)
                    match = pattern.search(full_text)
                    current_shared_paragraph = match.group(1).strip() if match else ""
                    current_range = r
                break

        choices_match = re.findall(
            r'①(.*?)②(.*?)③(.*?)④(.*?)⑤(.*?)(?=\n\d{1,2}[.)번]|\n\n|\Z)',
            body,
            re.DOTALL
        )
        if choices_match:
            choices = choices_match[0]
            question_match = re.split(r'①', body, maxsplit=1)
            question_text = question_match[0].strip()
            paragraph = current_shared_paragraph if in_shared_range else prev_text.strip()

            questions.append({
                "number": number,
                "paragraph": paragraph,
                "question": question_text,
                "choices": [o.strip() for o in choices]
            })

            prev_text = ""
        else:
            prev_text += "\n" + body

    return questions


def extract_common_subject_answers(pdf_path, max_questions=34):
    doc = fitz.open(pdf_path)
    text = doc[0].get_text()

    symbol_to_index = {'①': 0, '②': 1, '③': 2, '④': 3, '⑤': 4}
    matches = re.findall(r'(\d+)\s*([①-⑤])\s*\d+', text)

    answers = {}
    for number, symbol in matches:
        num = int(number)
        if 1 <= num <= max_questions:
            answers[num] = symbol_to_index[symbol]

    correct_indices = [answers[i] for i in range(1, max_questions + 1) if i in answers]
    
    if len(correct_indices) < max_questions:
        print(f"경고: 추출된 정답 수 {len(correct_indices)}개가 기대값 {max_questions}보다 적습니다.")

    return correct_indices

questions = extract_questions_from_pdf("Exam2.pdf")
answers = extract_common_subject_answers("Answer2.pdf")

import json

def print_questions(questions, max_items=None):
    if max_items is not None:
        questions = questions[:max_items]

    print(json.dumps(questions, ensure_ascii=False, indent=2))


# 정답 붙이기
for i in range(min(len(questions), len(answers))):
    questions[i]["answer"] = answers[i]

print (answers);
# 출력
print_questions(questions, max_items=34)


import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoModel

class ExamCrossAttention(nn.Module):
    def __init__(self, num_labels=5, hidden_size=768, num_heads=8):
        super().__init__()
        self.bert = AutoModel.from_pretrained("klue/bert-base")
        
        for param in self.bert.parameters():
            param.requires_grad = False
    
        self.cross_attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),

            nn.Linear(128, num_labels)
        )

        self.last_attn_weights = None
        
    def forward(self, input_ids_question, attention_mask_question, input_ids_paragraph, attention_mask_paragraph, labels=None):
        # 1. 질문 인코딩
        q_output = self.bert(input_ids=input_ids_question, attention_mask=attention_mask_question)
        question_embed = q_output.last_hidden_state  # (B, Lq, D)

        # 2. 지문 인코딩
        c_output = self.bert(input_ids=input_ids_paragraph, attention_mask=attention_mask_paragraph)
        paragraph_embed = c_output.last_hidden_state  # (B, Lc, D)

        # 3. Cross-Attention
        attn_output, attn_weights = self.cross_attention(
            query=question_embed,                # (B, Lq, D)
            key=paragraph_embed,                   # (B, Lc, D)
            value=paragraph_embed,                 # (B, Lc, D)
            key_padding_mask=(attention_mask_paragraph == 0),  # False = attend, True = mask
            average_attn_weights=False,
        )

        self.last_attn_weights = attn_weights.detach()
        
        # 4. Residual + LayerNorm
        attended = self.layer_norm(question_embed + attn_output)  # (B, Lq, D)

        # 5. 질문 토큰에 대한 pooling
        mean_pooled = attended.mean(dim=1)       # (B, D)
        max_pooled = attended.max(dim=1).values  # (B, D)

        pooled = (mean_pooled + max_pooled) / 2  # (B, D)


        logits = self.classifier(pooled)

        if labels is not None:
            labels_idx = labels.argmax(dim=1)
            loss = nn.CrossEntropyLoss()(logits, labels_idx)
            return {"loss": loss, "logits": logits}

        return {"logits": logits}
    


from torch.utils.data import Dataset

class MCQDataset(Dataset):
    def __init__(self, questions, tokenizer, max_q_len=512, max_p_len=512):
        self.questions = [q for q in questions if "answer" in q]
        self.tokenizer = tokenizer
        self.max_q_len = max_q_len
        self.max_p_len = max_p_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        q = self.questions[idx]
        question_text = q["question"]
        paragraph_text = q["paragraph"]  # 지문도 함께 사용
        label_idx = q["answer"]

        # 질문 인코딩
        q_encoding = self.tokenizer(
            question_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_q_len,
            return_tensors="pt"
        )

        # 지문 인코딩
        p_encoding = self.tokenizer(
            paragraph_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_p_len,
            return_tensors="pt"
        )

        # 레이블 (one-hot)
        one_hot = torch.zeros(5)
        one_hot[label_idx] = 1

        return {
            "input_ids_question": q_encoding["input_ids"].squeeze(0),
            "attention_mask_question": q_encoding["attention_mask"].squeeze(0),
            "input_ids_paragraph": p_encoding["input_ids"].squeeze(0),
            "attention_mask_paragraph": p_encoding["attention_mask"].squeeze(0),
            "labels": one_hot
        }


from transformers import TrainingArguments, Trainer

train_questions = []
test_questions = []

for i in range(1, 3):
    exam_file = f"Exam{i}.pdf"
    answer_file = f"Answer{i}.pdf"

    # 문제 및 정답 추출
    pdf_questions = extract_questions_from_pdf(exam_file)
    pdf_answers = extract_common_subject_answers(answer_file)

    for j in range(min(len(pdf_questions), len(pdf_answers))):
        pdf_questions[j]["answer"] = pdf_answers[j]

    train_questions.extend(pdf_questions)
    test_questions.extend(pdf_questions)
        


print(f"훈련용 문제 수: {len(train_questions)}")
print(f"테스트용 문제 수: {len(test_questions)}")

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
tokenizer.model_max_length = 512

for i in range(1, 3):
    exam_file = f"Exam{i}.pdf"
    answer_file = f"Answer{i}.pdf"

    questions = extract_questions_from_pdf(exam_file)
    answers = extract_common_subject_answers(answer_file)

    print(f"[Exam{i}] 문제 수: {len(questions)}, 정답 수: {len(answers)}")
    
# 학습용 Dataset 생성
train_dataset = MCQDataset(train_questions, tokenizer)

# 테스트용 Dataset 생성
test_dataset = MCQDataset(test_questions, tokenizer)

model = ExamCrossAttention()


training_args = TrainingArguments(
    output_dir="./Exam",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    eval_strategy="epoch",     
    save_strategy="epoch",           
    logging_strategy="epoch",
    load_best_model_at_end=True,     
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1
)


from transformers import TrainerCallback
from tqdm import tqdm

class LogAccuracyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        torch.save(model.state_dict(), "trained_model.pt")


trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[LogAccuracyCallback()]
)

import matplotlib.pyplot as plt

def plot_accuracy_from_trainer(trainer):
    log_history = trainer.state.log_history
    eval_steps = []
    eval_accs = []

    for entry in log_history:
        if "eval_accuracy" in entry:
            step = entry.get("epoch", entry.get("step", None))
            eval_steps.append(step)
            eval_accs.append(entry["eval_accuracy"])

    if not eval_accs:
        print("정확도 기록이 없습니다.")
        return

    plt.plot(eval_steps, eval_accs, marker='o', label="Eval Accuracy")
    plt.title("Evaluation Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

trainer.train()

model.load_state_dict(torch.load("trained_model.pt"))

from openai import OpenAI

client = OpenAI(api_key="")

def format_example_chunked_with_highlight(q):
    question = q["question"]
    paragraph = q["paragraph"]
    choices = q["choices"]

    # 1. 문단 나누기
    chunks = chunk_paragraph_by_sentences(paragraph)
    highlighted_chunks = []

    for i, chunk in enumerate(chunks):
        # highlight_words_with_cross_attention 사용
        try:
            highlighted, _ = highlight_words_with_cross_attention(
                question_text=question,
                paragraph_text=chunk,
                model=model,
                tokenizer=tokenizer,
                device="cuda"
            )
            highlighted_chunks.append(f"[문단 {i+1} 강조] {highlighted}")
        except Exception as e:
            print(f"[chunk {i}] 강조 실패: {e}")
            highlighted_chunks.append(f"[문단 {i+1}] {chunk}")

    # 2. 강조된 chunk들을 연결
    highlighted_summary = "\n\n".join(highlighted_chunks)

    return f"""[지문 요약 (강조 포함)] : 
{highlighted_summary}

[질문] : 
{question}

[선택지] : 
1) {choices[0]}
2) {choices[1]}
3) {choices[2]}
4) {choices[3]}
5) {choices[4]}

[정답] : 
{q['answer'] + 1}"""


url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
df = pd.read_json(url)

# 미리보기
df.head()


MAX_TOTAL_TOKENS = 2500

def count_tokens(text):
    return len(tokenizer.encode(text))

import torch
from transformers import DistilBertModel, DistilBertTokenizer

import string

def highlight_words_with_cross_attention(question_text, paragraph_text, model, tokenizer, max_q_len=512, max_p_len=512, top_k=20, device="cuda"):

    q_enc = tokenizer(question_text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_q_len)
    p_enc = tokenizer(paragraph_text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_p_len)

    input_ids_q = q_enc["input_ids"].to(device)
    mask_q = q_enc["attention_mask"].to(device)
    input_ids_p = p_enc["input_ids"].to(device)
    mask_p = p_enc["attention_mask"].to(device)

    with torch.no_grad():
        model.eval()
        _ = model(
            input_ids_question=input_ids_q,
            attention_mask_question=mask_q,
            input_ids_paragraph=input_ids_p,
            attention_mask_paragraph=mask_p
        )
    attn_weights = model.last_attn_weights  # (B, H, Lq, Lp)

    avg_attn = attn_weights.mean(dim=1)[0]     # (Lq, Lp)
    token_scores = avg_attn.mean(dim=0)        # (Lp,)

    tokens = tokenizer.convert_ids_to_tokens(input_ids_p[0], skip_special_tokens=True)

    words = []
    scores = []
    current_word = ""
    current_score_total = 0
    current_token_count = 0

    for token, score in zip(tokens, token_scores):
        if token.startswith("##"):
            token = token[2:]
            current_word += token
        else:
            if current_word:
                words.append(current_word)
                avg_score = current_score_total / max(current_token_count, 1)
                scores.append(avg_score)
            current_word = token
            current_score_total = 0
            current_token_count = 0
        current_score_total += score.item()
        current_token_count += 1

    if current_word:
        words.append(current_word)
        scores.append(current_score_total / max(current_token_count, 1))

    # top-k 단어 강조
    top_k = min(top_k, len(scores))
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

    highlighted_words = []
    for i, word in enumerate(words):
        # 문장 부호인지 확인
        if word in string.punctuation or word in ["``", "''", "”", "“", "’", "‘", "(", ")", ",", ".", "!", "?", "·", "…", ":", ";"]:
            # 문장 부호는 바로 앞 단어에 붙이기
            if highlighted_words:
                highlighted_words[-1] += word
            else:
                highlighted_words.append(word)
        else:
            if i in top_indices:
                highlighted_words.append(f"*{word}*")
            else:
                highlighted_words.append(word)

    highlighted_paragraph = " ".join(highlighted_words)
    return highlighted_paragraph, scores

from nltk.tokenize import sent_tokenize

def chunk_paragraph_by_sentences(paragraph, max_chunk_chars=200):
    sentences = sent_tokenize(paragraph)
    chunks = []
    current = ""

    for sent in sentences:
        if len(current) + len(sent) < max_chunk_chars:
            current += " " + sent
        else:
            chunks.append(current.strip())
            current = sent
    if current:
        chunks.append(current.strip())

    return chunks


def ask_gpt_with_chunked_summary_and_fewshot(problem, few_shot_pool, max_fewshots=1):
    import random

    paragraph = problem["paragraph"]
    q_text = problem["question"]
    if "question_plus" in problem:
        q_text += "\n\n" + problem["question_plus"]
    choices = problem["choices"]

    # 문단 쪼개기 + 강조 기반 요약
    chunks = chunk_paragraph_by_sentences(paragraph)
    context_summary = ""

    for i, chunk in enumerate(chunks):
        try:
            # 단어 강조 적용
            highlighted, _ = highlight_words_with_cross_attention(
                question_text=q_text,
                paragraph_text=chunk,
                model=model,
                tokenizer=tokenizer,
                device="cuda"
            )

            if i == 0:
                prompt = f"""
    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 핵심 내용을 간결하게 요약해 주세요. 중요한 정보는 생략하지 마세요.:

    [문단 {i+1}]
    {highlighted}
    """
            else:
                prompt = f"""
    이전까지의 요약:
    {context_summary}

    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 이전까지의 요약을 보완하거나 통합한 요약을 작성해 주세요. 이전까지의 요약 내용이 가능한 보존되어야 합니다.:

    [문단 {i+1}]
    {highlighted}
    """
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=4096,
            )
            context_summary = response.choices[0].message.content.strip()
            print(f"메시지1 : {prompt}")
            print(f"[요약 {i+1}] {context_summary}")

        except Exception as e:
            print(f"[chunk {i+1}] GPT 요약 오류: {e}")
            return -1


    few_shots = random.sample(few_shot_pool, k=min(max_fewshots, len(few_shot_pool)))
    few_shot_block = "\n\n---\n\n".join([format_example_chunked_with_highlight(q) for q in few_shots])

    choice_block = "\n".join([f"{i+1}) {c}" for i, c in enumerate(choices)])
    final_prompt = f"""
다음은 한국 수능형 객관식 문제입니다.
[지문]은 매우 길기 때문에 아래와 같이 요약되어 제공됩니다.

아래는 예시입니다:

{few_shot_block}

---

이제 새로운 문제입니다:

[지문 요약] :
{context_summary}

[질문] :
{q_text}

[선택지] :
{choice_block}

[정답] :
숫자 하나만 출력해주세요. 예: 3
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": final_prompt}],
            temperature=0,
            max_tokens=5,
        )
        result = response.choices[0].message.content.strip()
        match = re.search(r"\b([1-5])\b", result)
        print(f"메시지2 : {final_prompt}")
        print(f"\nGPT 최종 선택: {result}")
        return int(match.group(1)) if match else -1
    except Exception as e:
        print(f"GPT 오류: {e}")
        return -1


import random

# 전체 평가 루프
def evaluate_dataframe_with_few_shot(df, train_questions):
    correct = 0
    total_score = 0
    total_questions = 0

    train_questions = [q for q in train_questions if "answer" in q]

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Few-shot GPT 채점 중"):
        paragraph = row["paragraph"]
        problems = row["problems"]

        for i, problem in enumerate(problems):
            # 질문 내용 구성
            problem["paragraph"] = paragraph
            q_text = problem["question"]
            if "question_plus" in problem:
                q_text += "\n\n" + problem["question_plus"]

            # 예시 3개 가져오기
            few_shots = random.sample(train_questions, 3)
            if len(few_shots) < 3:
                continue  # 예시 부족하면 skip

            correct_answer = problem["answer"]
            score = problem.get("score", 1)

            print(f"\n[문제 {total_questions + 1}]")
            print(f"[질문] {q_text}")
            print(f"[정답] {correct_answer}")

            gpt_answer = ask_gpt_with_chunked_summary_and_fewshot(problem, few_shots)

            print(f"[GPT 선택] {gpt_answer}")

            if gpt_answer not in range(1, 6):
                print(f"GPT 정답 오류 : {gpt_answer}")
                
            if gpt_answer == correct_answer:
                correct += 1
                total_score += score
            total_questions += 1

    accuracy = correct / total_questions * 100 if total_questions > 0 else 0
    return accuracy, total_score, total_questions

# 채점 실행
accuracy, total_score, total_questions = evaluate_dataframe_with_few_shot(df, train_questions)

print(f"\n총 문제 수: {total_questions}")
print(f"정답 수: {int(accuracy * total_questions / 100)}")
print(f"정답률: {accuracy:.2f}%")
print(f"총 점수: {total_score}점")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[4, 1, 1, 0, 4, 1, 0, 3, 0, 2, 2, 3, 4, 3, 0, 0, 1, 3, 2, 2, 3, 0, 1, 4, 1, 4, 1, 2, 3, 2, 4, 3, 2, 2]
[
  {
    "number": 1,
    "paragraph": "다음 글을 읽고 물음에 답하시오.\n여러 글에서 다양한 정보를 종합하며 읽는 능력은 많은 정보가 \n산재해 있는 디지털 환경에서 더욱 중요해졌다. 궁금증 해소나 \n글쓰기 등 문제 해결을 위한 목적으로 글 읽기를 할 때에 한 \n편의 글에 원하는 정보가 충분하지 않다면, 여러 글을 읽으며 \n이를 해결할 수 있다.\n독자는 우선 문제 해결에 도움이 되는 글들을 찾아야 한다. \n읽을 글을 선정할 때에는 믿을 만한 글인지와 읽기 목적과 관련이 \n있는 글인지를 평가하는 것이 중요하다. ㉠신뢰성 평가는 글의 \n저자, 생산 기관, 출판 시기 등 출처에 관한 정보를 확인하여 \n그 글이 믿을 만한지 판단하는 것이다. ㉡관련성 평가는 글의 \n내용에 읽기 목적과 부합하는 정보가 있는지 판단하는 것인데, \n이를 위해서는 읽기 목적을 지속적으로 떠올리며 평가해 가야 \n한다.\n문제를 해결하기에 적절한 글들을 선정했다면, 다음으로는 \n읽기 목적에 맞게 글을 읽어야 한다. 이때 글의 정보는 독자가 \n이해한 의미로 재구성되고 이 과정에서 독자는 선택하기, 연결\n하기, 조직하기 전략을 활용한다. 이들 세 전략은 꼭 순서대로 \n사용하는 것은 아니며 반복해서 활용할 수 있다.\n선택하기란 읽은 글에서 필요한 정보를 추출하는 전략이다. \n연결하기란 읽은 글들에서 추출한 정보들을 정교화하며 연결하여, \n읽은 글에서는 나타나지 않던 의미를 구성하거나 심화된 의미로 \n나아가는 전략이다. 글의 정보를 재구조화하는 것은 조직하기\n라고 한다. 예를 들어, 시간의 순서에 따른 글과 정보 나열의 \n글을 읽고, 읽은 글의 구조와는 다른 비교․대조의 구조로 \n의미를 구성할 수 있다.\n이러한 전략을 적극적으로 활용하면, 정보의

훈련용 문제 수: 68
테스트용 문제 수: 68
[Exam1] 문제 수: 34, 정답 수: 34
[Exam2] 문제 수: 34, 정답 수: 34


Epoch,Training Loss,Validation Loss,Accuracy
1,1.641000,1.606862,0.250000
2,1.625600,1.589807,0.294118
3,1.606700,1.583043,0.367647
4,1.585600,1.576346,0.382353
5,1.582200,1.570216,0.485294
6,1.604800,1.566330,0.441176
7,1.563000,1.560240,0.529412
8,1.557100,1.551591,0.500000
9,1.586600,1.543223,0.544118
10,1.572900,1.535349,0.558824


Few-shot GPT 채점 중:   0%|          | 0/11 [00:00<?, ?it/s]

...

메시지1 : 
    이전까지의 요약:
    우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로, 커피에 대한 관심이 높습니다. 이러한 관심이 커피 사랑을 넘어 일회용 컵 회수 방안 같은 사회적 문제로 이어지는 것은 바람직합니다. 하지만 커피로 인한 사회적 문제 중 일부는 상대적으로 관심을 받지 못하고 있습니다. 특히, 커피를 만든 후 남는 커피 찌꺼기인 '커피박'에 대한 사회적 관심은 낮은 편입니다. 많은 사람들이 커피박을 잘못 처리하고 있으며, 추출 직후의 커피박을 싱크대 *배수구*나 *흙*에 버리는 등의 잘못된 방법을 사용하고 있습니다. *배수구에* 버린 *커피박에서* 나온 *카페인은* 하수 *처리* *과정에서* 완벽히 *걸러지지* 않은 채 *강물에* 흘러 들어가 *부정적으로* *작용할* 수 있으며, *흙에* 버린 *커피박은* 토양과 *식물에* 악영향을 줄 수 있습니다. *또한*, *커피박이* *다양한* *분야에서* *재활용될* 수 있다는 사실을 모르는 사람도 많습니다. *커피박은* 일상에서 *탈취제나* 방향제로 이용될 수 있습니다.

    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 이전까지의 요약을 보완하거나 통합한 요약을 작성해 주세요. 이전까지의 요약 내용이 가능한 보존되어야 합니다.:

    [문단 4]
    *그뿐만* 아니라 건축 *분야에서* 합성 *목재를* *대신하는* *재료로* 쓰이거나 농업 *분야에서* 혼합 *및* 발효 *과정을* 거쳐 *비료로* 사용되기도 한다. *최근에는* *바이오에너지의* *원료로* 활용 *될* *수* 있다는 점도 부각되고 있다. *끝으로*, *커피박* *수거* *시설이* 매우 부족하다는 점도 아쉬운 *부분이다*.
    
[요약 4] 우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로, 커피에 대한 관심이 높습니다. 이러한 관심이 커피 사랑을 넘어 일회용 컵 회수 방안 같은 사회적 문제로 이어지는 것은 바람직합니다. 그러나 커피로 인한 사회적 문제 중 

메시지1 : 
    이전까지의 요약:
    우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로, 커피에 대한 높은 관심이 일회용 컵 회수 방안 같은 사회적 문제로 이어지는 것은 긍정적이다. 그러나 커피로 인한 사회적 문제 중 일부는 상대적으로 관심을 받지 못하고 있다. 특히, 커피를 만든 후 남는 커피 찌꺼기인 '커피박'에 대한 사회적 관심은 낮은 편이다. 많은 사람들이 커피박을 잘못 처리하고 있으며, 추출 직후의 커피박을 싱크대 *배수구*나 *흙에* 버리는 등의 잘못된 방법을 사용하고 있다. *배수구에* 버린 커피박에서 *나온* *카페인은* 하수 *처리* *과정에서* 완벽히 *걸러지지* 않은 채 *강물에* 흘러 들어가 부정적으로 *작용할* 수 있으며, *흙에* 버린 커피박은 토양과 식물에 악영향을 줄 수 있다. *또한*, 커피박이 *다양한* *분야에서* *재활용될* 수 있다는 사실을 모르는 사람도 많다. 커피박은 *일상에서* *탈취제나* 방향제로 이용될 수 있다.

    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 이전까지의 요약을 보완하거나 통합한 요약을 작성해 주세요. 이전까지의 요약 내용이 가능한 보존되어야 합니다.:

    [문단 4]
    *그뿐만* 아니라 건축 *분야에서* 합성 *목재를* *대신하는* *재료로* 쓰이거나 농업 *분야에서* 혼합 *및* 발효 *과정을* 거쳐 *비료로* 사용되기도 한다. *최근에는* *바이오에너지의* *원료로* 활용 될 *수* 있다는 점도 부각되고 있다. *끝으로*, *커피박* *수거* *시설이* 매우 부족하다는 점도 아쉬운 *부분이다*.
    
[요약 4] 우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로, 커피에 대한 높은 관심이 일회용 컵 회수 방안 같은 사회적 문제로 이어지는 것은 긍정적이다. 그러나 커피로 인한 사회적 문제 중 일부는 상대적으로 관심을 받지 못하고 있다. 특히, 커피를 만든 후 남는 커피 찌꺼기인 '커피박'에 대한 사회적 관심

메시지1 : 
    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 핵심 내용을 간결하게 요약해 주세요. 중요한 정보는 생략하지 마세요.:

    [문단 1]
    *우리나라의* 연간 1인당 *커피* *소비량은* 세계 *평균의* 2배 *이상일* 정도로 우리나라 사람들은 커피를 마시는 일에 관심이 많다. *이러한* *관심이* 커피 *사랑에만* 머물지 않고, *일회용* *컵* *회수* *방안* *처럼* *커피로* 인한 사회적 문제에 대한 관심으로 이어지는 현상은 *바람직하다*. *하지만* *커피로* 인한 사회적 문제를 논할 *때*, *상대적* 으로 *관심을* 받지 못하고 있는 것이 있다.
    
[요약 1] 우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로 높으며, 이는 커피에 대한 높은 관심을 반영합니다. 이러한 관심이 커피 사랑을 넘어 일회용 컵 회수 방안 등 커피로 인한 사회적 문제로 이어지는 것은 바람직합니다. 그러나 커피로 인한 사회적 문제 중 일부는 상대적으로 관심을 받지 못하고 있습니다.
메시지1 : 
    이전까지의 요약:
    우리나라의 연간 1인당 커피 소비량은 세계 평균의 2배 이상으로 높으며, 이는 커피에 대한 높은 관심을 반영합니다. 이러한 관심이 커피 사랑을 넘어 일회용 컵 회수 방안 등 커피로 인한 사회적 문제로 이어지는 것은 바람직합니다. 그러나 커피로 인한 사회적 문제 중 일부는 상대적으로 관심을 받지 못하고 있습니다.

    아래 문단은 중요한 단어가 *강조*되어 있습니다.
    이 정보를 반영하여 이전까지의 요약을 보완하거나 통합한 요약을 작성해 주세요. 이전까지의 요약 내용이 가능한 보존되어야 합니다.:

    [문단 2]
    *커피를* 만든 *후* 남는 커피 찌꺼기, 바로‘ 커피박( coffee)’ *이다*. *여러* *면에서* 커피박에 *대한* 우리 사회의 관심은 낮은 편이다. 우선, *커피박을* *잘못* 처리하고 *있는* 사람이 많다. 추출 *직후* *의* *커

Few-shot GPT 채점 중: 100%|██████████| 11/11 [41:59<00:00, 229.05s/it]

메시지2 : 
다음은 한국 수능형 객관식 문제입니다.
[지문]은 매우 길기 때문에 아래와 같이 요약되어 제공됩니다.

아래는 예시입니다:

[지문 요약 (강조 포함)] : 
[문단 1 강조] *다음* 글을 읽고 물음에 답하시오.[ 앞부분의 줄거리] 해방 *직후*, 미군 *소위의* *통역을* 맡아 부정 *축재를* 일삼던 *방삼복은* 고향에서 온 백 *주사를* 집으로 초대한다.“ 서 *주사가* 이거 *두구* *갑디다*.” 들고 올라온 *각봉투* *한* 장을 남편에게 건네어 준다.“ 어디?” 그러면서 받아 *봉을* 뜯는다. *소절수* *한* 장이 나온다. 액면 *만* 원짜리다.

[문단 2 강조] *미스터* *방은* 성을 벌컥 내면서“ 겨우 *둔* *만* 원야?” 하고 소절수를 다다미 바닥에다 내던진다.“ *내가* 알우?”“ 우랄질 *자식* *어디* *보자*. 그래 전, *걸* *십만* *원에* *불하* *맡아다*, 백만 원 *하난* *냉겨* *먹을* 테문서, 그래 겨우 둔 *만* 원야?

[문단 3 강조] *엠병헐* *자식*, 엠피* *헌테* *말* 한마디문, *전* 어느 지경 갈지 *모를* 줄 모르구서.”“ 정종으루 가져와요?”“ 내 말 한마디에, 죽을 눔이 *살아나구*, 살 눔이 죽구 허는 줄은 모르구서. *흥*, 이 자식 경 *좀* *쳐* 봐라……. 증종 *따근허게* *데와*. 날두 *산산허구* *허니*.” 새로이 안주가 오고, 따끈한 정종으로 술이 몇 잔 더 오락가락 하고 나서였다.

[문단 4 강조] *백* *주사는* 마침내, *진작부터* 벼르던 이야기를 꺼내었다. *백* *주사의* 아들, *순사* *임명장을* 받아 쥐면서부터 시작하여 8ㆍ15 *그* 전날까지 칠 년 *동안*, 세 곳 *주재소와* 두 곳 경찰서를 *전근하여* *다니면서*, 이백 *석* *추수의* *토지와*, 만 원짜리 *저금통장과*, 만 원어치가 넘는 옷이며 비단과, 역시 만 *원어치가* 넘는 여편네의 *패물과를* 장만하였다.

[문단 5 강조] *남들은* *주린* *창자를* 때 *그의*